In [20]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

df=pd.read_csv('Meteorite_Landings.csv')

# Initial cleanup by removing rows with any missing values
clean_df = df.dropna()

# Filtering the dataset for meteorites that landed within the year range 1756 to 1963
filtered_df = clean_df[(clean_df['year'] >= 1756) & (clean_df['year'] <= 1963)]

# Removing the nametype, id. recclass
filtered_df_dropped = filtered_df.drop(columns=['nametype', 'id', 'recclass'])

# Saving the modified dataset to a new CSV file
filtered_df_dropped.to_csv('Cleaned_Meteorite_Landings.csv', index=False)

#Filtering by reclat and reclong to only have north america locations
north_america_df = filtered_df_dropped[(filtered_df_dropped['reclat'] >= 15) & 
                                       (filtered_df_dropped['reclat'] <= 60) & 
                                       (filtered_df_dropped['reclong'] >= -168) & 
                                       (filtered_df_dropped['reclong'] <= -50)]

north_america_df.to_csv('Cleaned_Meteorite_Landings.csv', index=False)

print('Cleaned_Meteorite_Landings.csv')



Cleaned_Meteorite_Landings.csv


In [21]:
##test section for first 10 rows
# df = pd.read_csv('Cleaned_Meteorite_Landings.csv', nrows=10)
# geolocator = Nominatim(user_agent="meteorite_locator")

# # using geolocation to match states to the lat/long
# def get_state(geolocation):
#     # parse GeoLocation column for latitude and longitude
#     lat, lon = map(str.strip, geolocation.strip('()').split(','))
#     # passes coordinates to geocoder
#     location = geolocator.reverse((lat, lon), exactly_one=True)
#     if location:
#         address = location.raw['address']
#         return address.get('state', 'Unknown')
#     else:
#         return 'Unknown'

# # apply to every row
# df['State'] = df['GeoLocation'].apply(get_state)

# # writes test CSV
# df.to_csv('meteorites_with_state_test.csv', index=False)

In [22]:

##test section to create state column for entire dataset

# df = pd.read_csv('Cleaned_Meteorite_Landings.csv')
# geolocator = Nominatim(user_agent="meteorite_locator")

# # five second delay between requests bc it kept timing out

# time.sleep(5)

# def get_state(geolocation):
#     lat, lon = map(str.strip, geolocation.strip('()').split(','))
#     location = geolocator.reverse((lat, lon), exactly_one=True)
#     if location:
#         address = location.raw['address']
#         return address.get('state', 'Unknown')
#     else:
#         return 'Unknown'

# df['State'] = df['GeoLocation'].apply(get_state)

# df.to_csv('meteorites_with_state.csv', index=False)


In [23]:

#this one writes a column for state and also creates a column for country
df = pd.read_csv('Cleaned_Meteorite_Landings.csv')
geolocator = Nominatim(user_agent="meteorite_locator")
time.sleep(5)

def get_state_and_country(geolocation):
    try:
        lat, lon = map(str.strip, geolocation.strip('()').split(','))
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location:
            address = location.raw['address']
            state = address.get('state', 'Unknown')
            country = address.get('country', 'Unknown')
            return state, country
        else:
            return 'Unknown', 'Unknown'
    except GeocoderTimedOut:
        return 'TimedOut', 'TimedOut'

df['State'], df['Country'] = zip(*df['GeoLocation'].apply(get_state_and_country))

df.to_csv('meteorites_with_state_and_country.csv', index=False)
